## 1 (改訂 v2). ライブラリのインポートと設定 (最大量取得・プログレスバー対応)
データ取得上限を増やし、プログレスバー表示のための tqdm をインポートします。
tqdm が未インストールの場合: !pip install tqdm
`start_date_str` で取得を開始したい最も古い日付を指定します。
`max_total_data` で取得するデータ件数の大まかな上限を設定できます（メモリ保護のため）。


In [1]:
import pybotters
import pandas as pd
import asyncio
import time
from datetime import datetime, timedelta, timezone
import nest_asyncio
from tqdm.notebook import tqdm # Jupyter Notebook 用の tqdm をインポート

# --- 設定項目 ---
target_symbol = 'BTCUSDT'
interval = '5'
category = 'linear'
limit = 1000
start_date_str = '2000-01-01' # 取得開始希望日をさらに過去に設定 (例: 2020年)
# max_total_data = 1000 # 取得上限を増やす (例: 100万件)
max_total_data = None      # または None にしてAPIが提供する限り取得する (メモリ注意)
output_filename_full = f'{target_symbol}_{interval}m_data_max.csv' # ファイル名変更

# --- 設定項目ここまで ---

base_url = 'https://api.bybit.com'
start_timestamp_ms = int(datetime.strptime(start_date_str, '%Y-%m-%d').replace(tzinfo=timezone.utc).timestamp() * 1000)


## 2 (改訂 v2). データ取得・整形関数の定義 (最大量取得・プログレスバー版)
指定した開始日まで遡ってデータを取得するよう修正した関数。
tqdm を組み込み、進捗を表示します。

In [2]:
async def fetch_bybit_kline_full_tqdm(symbol, interval, category, limit, start_ts_ms, max_data=None):
    """Bybit V5 APIから指定開始日まで遡ってデータを取得し、DataFrameに整形 (tqdm進捗表示付き)"""
    apis = {}
    client = pybotters.Client(apis=apis, base_url=base_url)
    endpoint = '/v5/market/kline'

    all_data_list = []
    current_end_time = int(time.time() * 1000)
    total_fetched = 0
    request_count = 0
    estimated_total_requests = None # 総リクエスト回数の推定値 (任意)

    # 大まかな総リクエスト回数を推定する場合（任意、正確ではない）
    if max_data:
         estimated_total_requests = (max_data // limit) + 1
    else:
         # 現在から開始日までのおおよその期間を計算して推定 (ざっくり)
         try:
             duration_days = (datetime.now(timezone.utc) - datetime.fromtimestamp(start_ts_ms / 1000, tz=timezone.utc)).days
             estimated_total_requests = (duration_days * 24 * (60 // int(interval)) // limit) + 5 # 余裕を持たせる
             print(f"推定総リクエスト回数 (目安): {estimated_total_requests}")
         except:
             pass # 計算失敗しても気にしない

    print(f"データ取得を開始します (開始希望日: {start_date_str})...")

    # tqdmの初期化 (totalが不明な場合もある)
    # descで何のプログレスバーか示す, unit='req' で単位をリクエストにする
    pbar = tqdm(total=estimated_total_requests, desc=f"Fetching {symbol}", unit="req")

    while True:
        request_count += 1
        params = {
            'category': category,
            'symbol': symbol,
            'interval': interval,
            'limit': limit,
            'end': current_end_time,
        }
        try:
            resp = await client.get(endpoint, params=params)
            data = await resp.json()

            if data['retCode'] == 0 and data['result'] and data['result']['list']:
                kline_list = data['result']['list']
                fetched_count = len(kline_list)
                total_fetched += fetched_count
                oldest_timestamp_in_batch = int(kline_list[-1][0])

                # tqdmの進捗を更新 (取得件数も表示させる postifx)
                pbar.update(1)
                pbar.set_postfix(fetched=f"{total_fetched/1000:.1f}k", last_dt=f"{datetime.fromtimestamp(oldest_timestamp_in_batch / 1000, tz=timezone.utc).strftime('%Y-%m-%d')}")

                all_data_list.extend(kline_list)

                if oldest_timestamp_in_batch <= start_ts_ms or fetched_count < limit:
                    print("\n目標開始日以前のデータに到達、または取得データがlimit未満になったため終了します。")
                    break
                if max_data is not None and total_fetched >= max_data:
                    print(f"\n最大取得件数 ({max_data} 件) に到達したため終了します。")
                    break

                current_end_time = oldest_timestamp_in_batch - 1
                await asyncio.sleep(0.2) # レートリミット考慮

            else:
                print(f"\nReq {request_count}: データ取得エラーまたはデータがありません。Response: {data}")
                break # ループ終了

        except Exception as e:
            print(f"\nReq {request_count}: リクエスト中にエラーが発生しました: {e}")
            await asyncio.sleep(1)
            if request_count > 5 and total_fetched == 0:
                 print("\n初期のデータ取得でエラーが続いたため中断します。")
                 break
            continue

    pbar.close() # プログレスバーを閉じる

    if not all_data_list:
        print("有効なデータを取得できませんでした。")
        return pd.DataFrame()

    # --- DataFrame変換以降は同じ ---
    print("\nDataFrame変換中...")
    df = pd.DataFrame(all_data_list, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
    df.set_index('timestamp', inplace=True)
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')
    df = df[~df.index.duplicated(keep='first')]
    df.sort_index(ascending=True, inplace=True)
    df = df[df.index >= pd.Timestamp(start_date_str, tz='UTC')]
    print("DataFrame変換完了。")

    print(f"\nデータ取得完了。合計 {len(df)} 件の一意なデータを取得しました。")
    return df


## 3 (改訂 v2). データ取得の実行と結果確認 (最大量取得・プログレスバー版)

In [3]:
nest_asyncio.apply() # Jupyter環境用

if __name__ == '__main__':
    try:
        print("最大量のデータ取得処理を開始します...(プログレスバーが表示されます)")
        # 関数名を変更したものを呼び出す
        df_kline_max = asyncio.run(fetch_bybit_kline_full_tqdm(target_symbol, interval, category, limit, start_timestamp_ms, max_total_data))
        print("データ取得処理が完了しました。")
    except Exception as e:
        print(f"データ取得中に予期せぬエラーが発生しました: {e}")
        df_kline_max = pd.DataFrame() # エラー時は空のDataFrame

# --- 結果表示 (変数名を df_kline_max に変更) ---
if not df_kline_max.empty:
    print("\n--- 取得データ (最初の5行) ---")
    display(df_kline_max.head())
    print("\n--- 取得データ (最後の5行) ---")
    display(df_kline_max.tail())
    print("\n--- データフレーム情報 ---")
    df_kline_max.info()
    print(f"\n取得期間: {df_kline_max.index.min()} ~ {df_kline_max.index.max()}")
    print(f"データ件数: {len(df_kline_max)}")

    # (任意) 取得したデータを保存
    try:
        print(f"\nデータを '{output_filename_full}' として保存中...")
        df_kline_max.to_csv(output_filename_full)
        print("保存完了。")
    except Exception as e:
        print(f"\nデータの保存中にエラーが発生しました: {e}")
else:
    print("\nデータフレームが空、または取得に失敗しました。")


最大量のデータ取得処理を開始します...(プログレスバーが表示されます)
推定総リクエスト回数 (目安): 2666
データ取得を開始します (開始希望日: 2000-01-01)...


Fetching BTCUSDT:   0%|          | 0/2666 [00:00<?, ?req/s]


目標開始日以前のデータに到達、または取得データがlimit未満になったため終了します。

DataFrame変換中...


C:\Users\trow1\AppData\Local\Temp\ipykernel_69492\3035594412.py:87: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True)
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000020105AE4980>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x0000020105B6C890>, 1020688.9855927)])']
connector: <aiohttp.connector.TCPConnector object at 0x0000020105AE4AD0>


DataFrame変換完了。

データ取得完了。合計 533705 件の一意なデータを取得しました。
データ取得処理が完了しました。

--- 取得データ (最初の5行) ---


,open,high,low,close,volume,turnover
timestamp,,,,,,
2020-03-25 10:35:00+00:00,6500.0,6500.0,6500.0,6500.0,0.001,6.5000
2020-03-25 10:40:00+00:00,6500.0,6500.0,6500.0,6500.0,0.001,6.5000
2020-03-25 10:45:00+00:00,6500.0,6500.0,6500.0,6500.0,0.000,0.0000
2020-03-25 10:50:00+00:00,6500.0,6588.0,6500.0,6588.0,0.001,6.5880
2020-03-25 10:55:00+00:00,6588.0,6591.5,6588.0,6591.5,0.001,6.5915



--- 取得データ (最後の5行) ---


,open,high,low,close,volume,turnover
timestamp,,,,,,
2025-04-21 13:35:00+00:00,87320.5,87500.0,86884.2,87478.1,1359.941,1.185635e+08
2025-04-21 13:40:00+00:00,87478.1,87570.0,87300.0,87399.2,1245.267,1.088904e+08
2025-04-21 13:45:00+00:00,87399.2,87485.0,87187.9,87447.0,778.859,6.804919e+07
2025-04-21 13:50:00+00:00,87447.0,87478.2,87231.4,87235.9,371.193,3.243191e+07
2025-04-21 13:55:00+00:00,87235.9,87261.6,87008.7,87037.3,477.574,4.159717e+07



--- データフレーム情報 ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 533705 entries, 2020-03-25 10:35:00+00:00 to 2025-04-21 13:55:00+00:00
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   open      533705 non-null  float64
 1   high      533705 non-null  float64
 2   low       533705 non-null  float64
 3   close     533705 non-null  float64
 4   volume    533705 non-null  float64
 5   turnover  533705 non-null  float64
dtypes: float64(6)
memory usage: 28.5 MB

取得期間: 2020-03-25 10:35:00+00:00 ~ 2025-04-21 13:55:00+00:00
データ件数: 533705

データを 'BTCUSDT_5m_data_max.csv' として保存中...
保存完了。


## 特徴量セット A (WclPriceベース) の作成

Weighted Close (WclPrice) を計算し、主要なテクニカル指標の計算に WclPrice を使用します。
tqdm で進捗を表示し、目的変数は時間差チェック済みのものを利用します。

In [4]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from tqdm.notebook import tqdm

# df_kline_max (最大量データ) がロード済みとする
if 'df_kline_max' in locals() and not df_kline_max.empty:
    print("特徴量セット A (WclPriceベース) の作成を開始します...")
    df_features_wcl = df_kline_max.copy()

    # 1. Weighted Close (WclPrice) の計算
    print("Calculating WclPrice...")
    df_features_wcl['wclprice'] = (df_features_wcl['high'] + df_features_wcl['low'] + 2 * df_features_wcl['close']) / 4

    # --- 特徴量計算 (WclPrice を入力に使用) ---
    # 2. 基本的な価格特徴量 (WclPrice基準) - オプション
    df_features_wcl['open_norm_wcl'] = df_features_wcl['open'] / df_features_wcl['wclprice']
    df_features_wcl['high_norm_wcl'] = df_features_wcl['high'] / df_features_wcl['wclprice']
    df_features_wcl['low_norm_wcl'] = df_features_wcl['low'] / df_features_wcl['wclprice']
    df_features_wcl['close_norm_wcl'] = df_features_wcl['close'] / df_features_wcl['wclprice']

    # 3. テクニカル指標 (入力に 'wclprice' を指定)
    print("テクニカル指標 (WclPriceベース) を計算中...")
    periods_ma = [7, 14, 21, 50, 100, 200]
    print("Calculating MAs (WclPrice)...")
    for length in tqdm(periods_ma, desc="MAs_WCL"):
        # pandas-ta で入力列を指定するには close='wclprice' のように引数を渡す
        df_features_wcl.ta.sma(close=df_features_wcl['wclprice'], length=length, append=True, col_names=(f'SMA_{length}_WCL'))
        df_features_wcl.ta.ema(close=df_features_wcl['wclprice'], length=length, append=True, col_names=(f'EMA_{length}_WCL'))

    print("Calculating MACD (WclPrice)...")
    df_features_wcl.ta.macd(close=df_features_wcl['wclprice'], append=True, col_names=('MACD_WCL', 'MACDh_WCL', 'MACDs_WCL')) # 列名を変更

    periods_rsi = [7, 14, 21]
    print("Calculating RSI (WclPrice)...")
    for length in tqdm(periods_rsi, desc="RSI_WCL"):
         df_features_wcl.ta.rsi(close=df_features_wcl['wclprice'], length=length, append=True, col_names=(f'RSI_{length}_WCL'))

    # Stochastics は High, Low, Close を使うので WclPrice ではなく通常通り計算
    print("Calculating Stochastics...")
    df_features_wcl.ta.stoch(append=True) # STOCHk_14_3_3, STOCHd_14_3_3

    # StochRSI は RSI に基づくので、WclPrice ベースの RSI を使うか検討 -> ここでは Close ベースの RSI を使う StochRSI を計算
    print("Calculating StochRSI (Close base)...")
    df_features_wcl.ta.stochrsi(append=True) # STOCHRSIk_14_14_3_3, STOCHRSId_14_14_3_3

    # Bollinger Bands (WclPrice ベース)
    print("Calculating Bollinger Bands (WclPrice)...")
    df_features_wcl.ta.bbands(close=df_features_wcl['wclprice'], length=20, std=2, append=True, col_names=('BBL_WCL', 'BBM_WCL', 'BBU_WCL', 'BBB_WCL', 'BBP_WCL')) # 列名変更

    # ATR (通常通り H, L, C を使う)
    print("Calculating ATR...")
    df_features_wcl.ta.atr(length=14, append=True, col_names=('ATR_14'))

    # ADX (通常通り H, L, C を使う)
    print("Calculating ADX...")
    df_features_wcl.ta.adx(length=14, append=True) # ADX_14, DMP_14, DMN_14

    # CCI (WclPrice ベースで計算可能か？ -> HLC を使うのが一般的) -> 通常通り計算
    print("Calculating CCI (HLC base)...")
    df_features_wcl.ta.cci(length=14, append=True, col_names=('CCI_14'))

    # Williams %R (通常通り H, L, C を使う)
    print("Calculating Williams %R...")
    df_features_wcl.ta.willr(length=14, append=True, col_names=('WILLR_14'))

    # OBV (通常通り C, V を使う)
    print("Calculating OBV...")
    df_features_wcl.ta.obv(append=True)

    # --- 4. ラグ特徴量 (WclPrice と Close の両方) ---
    print("ラグ特徴量を計算中...")
    periods_return = [1, 2, 3, 5, 10, 20, 50]
    for n in tqdm(periods_return, desc="Returns"):
        df_features_wcl[f'return_{n}'] = df_features_wcl['close'].pct_change(periods=n)
        df_features_wcl[f'return_{n}_wcl'] = df_features_wcl['wclprice'].pct_change(periods=n) # WclPriceのリターンも追加

    # --- 5. 時間特徴量 ---
    print("時間特徴量を計算中...")
    df_features_wcl['hour'] = df_features_wcl.index.hour
    df_features_wcl['dayofweek'] = df_features_wcl.index.dayofweek

    # --- 6. 目的変数作成 (時間差チェック付き) ---
    print("目的変数 (ターゲット) を作成中 (5分間隔チェック付き)...")
    df_features_wcl['timediff'] = df_features_wcl.index.to_series().diff()
    df_features_wcl['next_close'] = df_features_wcl['close'].shift(-1)
    df_features_wcl['next_timediff'] = df_features_wcl['timediff'].shift(-1)
    condition_high = (df_features_wcl['next_close'] > df_features_wcl['close']) & (df_features_wcl['next_timediff'] == pd.Timedelta('5 minutes'))
    condition_low = (df_features_wcl['next_close'] <= df_features_wcl['close']) & (df_features_wcl['next_timediff'] == pd.Timedelta('5 minutes'))
    df_features_wcl['target'] = np.select([condition_high, condition_low], [1.0, 0.0], default=np.nan)
    df_features_wcl = df_features_wcl.drop(columns=['timediff', 'next_timediff', 'next_close'])

    # --- 7. NaN削除 ---
    rows_before_dropna = len(df_features_wcl)
    print(f"\nNaN削除前の行数: {rows_before_dropna}")
    print("NaN削除処理を開始します...")
    df_processed_wcl = df_features_wcl.dropna()
    print("NaN削除処理完了。")
    rows_after_dropna = len(df_processed_wcl)
    if 'target' in df_processed_wcl.columns:
         df_processed_wcl['target'] = df_processed_wcl['target'].astype(int)

    print(f"NaN削除後の行数: {rows_after_dropna}")
    print(f"削除された行数: {rows_before_dropna - rows_after_dropna}")

    print("\n--- 特徴量セット A (WclPrice) 作成完了 ---")
    df_processed_wcl.info(verbose=False, memory_usage='deep')
    display(df_processed_wcl.head())

    # (任意) 保存
    df_processed_wcl.to_csv('processed_data_wcl.csv')

else:
    print("df_kline_max が存在しないか空です。データ取得ステップを先に実行してください。")


特徴量セット A (WclPriceベース) の作成を開始します...
Calculating WclPrice...
テクニカル指標 (WclPriceベース) を計算中...
Calculating MAs (WclPrice)...


MAs_WCL:   0%|          | 0/6 [00:00<?, ?it/s]

Calculating MACD (WclPrice)...
Calculating RSI (WclPrice)...


RSI_WCL:   0%|          | 0/3 [00:00<?, ?it/s]

Calculating Stochastics...
Calculating StochRSI (Close base)...
Calculating Bollinger Bands (WclPrice)...
Calculating ATR...
Calculating ADX...
Calculating CCI (HLC base)...
Calculating Williams %R...
Calculating OBV...
ラグ特徴量を計算中...


Returns:   0%|          | 0/7 [00:00<?, ?it/s]

時間特徴量を計算中...
目的変数 (ターゲット) を作成中 (5分間隔チェック付き)...

NaN削除前の行数: 533705
NaN削除処理を開始します...
NaN削除処理完了。
NaN削除後の行数: 533505
削除された行数: 200

--- 特徴量セット A (WclPrice) 作成完了 ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 533505 entries, 2020-03-26 03:10:00+00:00 to 2025-04-21 13:50:00+00:00
Columns: 62 entries, open to target
dtypes: float64(59), int32(2), int64(1)
memory usage: 252.4 MB


C:\Users\trow1\AppData\Local\Temp\ipykernel_69492\2216702590.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed_wcl['target'] = df_processed_wcl['target'].astype(int)


,open,high,low,close,volume,turnover,wclprice,open_norm_wcl,high_norm_wcl,low_norm_wcl,...,return_5_wcl,return_10,return_10_wcl,return_20,return_20_wcl,return_50,return_50_wcl,hour,dayofweek,target
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-03-26 03:10:00+00:00,6687.5,6687.5,6667.5,6670.0,35.519,236911.730,6673.750,1.002060,1.002060,0.999063,...,-0.001944,-0.000450,0.000300,-0.005517,-0.004475,-0.001048,-0.000636,3,3,1
2020-03-26 03:15:00+00:00,6670.0,6693.5,6670.0,6693.5,92.544,619443.264,6687.625,0.997365,1.000878,0.997365,...,-0.000766,0.004728,0.003432,-0.001864,-0.002424,0.003598,0.002567,3,3,0
2020-03-26 03:20:00+00:00,6693.5,6693.5,6682.5,6682.5,10.882,72718.965,6685.250,1.001234,1.001234,0.999589,...,-0.001307,0.000749,0.002005,-0.004840,-0.003707,0.004283,0.004112,3,3,1
2020-03-26 03:25:00+00:00,6682.5,6694.0,6682.5,6685.0,11.240,75139.400,6686.625,0.999383,1.001103,0.999383,...,0.000355,0.000749,0.001085,-0.003057,-0.003168,0.003528,0.004054,3,3,1
2020-03-26 03:30:00+00:00,6685.0,6738.0,6685.0,6737.5,13.516,91064.050,6724.500,0.994126,1.002008,0.994126,...,0.005928,0.008608,0.006662,0.007100,0.004575,0.010120,0.008492,3,3,0


## 特徴量セット B (ADR/PowerX要素追加) の作成

特徴量セットA (WclPriceベース、NaN削除前) に、ATR(7) と PowerX戦略で使われる指標パラメータ (Closeベース) を追加します。

In [5]:
import pandas as pd
import pandas_ta as ta
import numpy as np
from tqdm.notebook import tqdm

# 前のステップで作成した df_features_wcl (特徴量セットA、NaN削除前) が存在すると仮定
if 'df_features_wcl' in locals() and not df_features_wcl.empty:
    print("特徴量セット B (ADR/PowerX要素追加) の作成を開始します...")
    # Set A (NaN削除前) をコピーして開始
    df_features_b = df_features_wcl.copy()

    # --- ADR (ATRで代用) の追加 ---
    print("Calculating ATR(7)...")
    # ATRは High, Low, Close を使う
    # length=7 を指定
    df_features_b.ta.atr(length=7, append=True, col_names=('ATR_7'))

    # --- PowerX 指標パラメータの確認・追加 (Closeベース) ---
    # RSI(7) - Closeベース
    if 'RSI_7' not in df_features_b.columns: # CloseベースのRSI(7)がなければ追加
        print("Calculating RSI(7) - Close base...")
        df_features_b.ta.rsi(length=7, append=True, col_names=('RSI_7'))
    else:
        print("RSI(7) - Close base already exists.")

    # Stochastics(14, 3, 3) - Closeベース (通常はこちらがデフォルトのはず)
    # Set A 作成時に追加済みのはずなので確認のみ
    if 'STOCHk_14_3_3' in df_features_b.columns and 'STOCHd_14_3_3' in df_features_b.columns:
        print("Stochastics(14,3,3) - Close base already exists.")
    else:
        print("Warning: Stochastics(14,3,3) not found, consider adding it based on Close.")
        # df_features_b.ta.stoch(k=14, d=3, smooth_k=3, append=True) # 必要なら追加

    # MACD(12, 26, 9) - Closeベース
    if 'MACD_12_26_9' not in df_features_b.columns: # CloseベースのMACDがなければ追加
         print("Calculating MACD(12,26,9) - Close base...")
         # pandas-ta のデフォルトが (12, 26, 9) なので、引数なしでも良いはず
         df_features_b.ta.macd(append=True) # これで MACD_12_26_9, MACDh_12_26_9, MACDs_12_26_9 が追加されるはず
    else:
         print("MACD(12,26,9) - Close base already exists.")

    # --- 目的変数作成 (変更なし) ---
    # target 列は df_features_wcl に既に存在し、NaNが含まれているはず
    print("Target variable already exists (incl. NaNs from time check).")


    # --- NaN削除 ---
    rows_before_dropna_b = len(df_features_b)
    print(f"\nNaN削除前の行数 (Set B): {rows_before_dropna_b}")
    print("NaN削除処理を開始します...")
    # 追加した指標によって NaN が発生する行が増える可能性がある
    df_processed_b = df_features_b.dropna()
    print("NaN削除処理完了。")
    rows_after_dropna_b = len(df_processed_b)
    if 'target' in df_processed_b.columns:
         # target列を整数型に変換
         df_processed_b['target'] = df_processed_b['target'].astype(int)

    print(f"NaN削除後の行数 (Set B): {rows_after_dropna_b}")
    print(f"削除された行数: {rows_before_dropna_b - rows_after_dropna_b}")

    print("\n--- 特徴量セット B (ADR/PowerX追加) 作成完了 ---")
    print(f"最終的な特徴量の数: {len(df_processed_b.columns) - 1}") # target除く
    df_processed_b.info(verbose=False, memory_usage='deep')
    display(df_processed_b.head())

    # (任意) 保存
    output_processed_filename_b = f'{target_symbol}_{interval}m_processed_set_b.csv'
    try:
        print(f"\n処理済みデータを {output_processed_filename_b} として保存中...")
        df_processed_b.to_csv(output_processed_filename_b)
        print("保存完了。")
    except Exception as e:
        print(f"\nデータの保存中にエラーが発生しました: {e}")

else:
    print("df_features_wcl (特徴量セットA, NaN削除前) が見つかりません。")

特徴量セット B (ADR/PowerX要素追加) の作成を開始します...
Calculating ATR(7)...
Calculating RSI(7) - Close base...
Stochastics(14,3,3) - Close base already exists.
Calculating MACD(12,26,9) - Close base...
Target variable already exists (incl. NaNs from time check).

NaN削除前の行数 (Set B): 533705
NaN削除処理を開始します...
NaN削除処理完了。
NaN削除後の行数 (Set B): 533505
削除された行数: 200

--- 特徴量セット B (ADR/PowerX追加) 作成完了 ---
最終的な特徴量の数: 66
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 533505 entries, 2020-03-26 03:10:00+00:00 to 2025-04-21 13:50:00+00:00
Columns: 67 entries, open to MACDs_12_26_9
dtypes: float64(64), int32(2), int64(1)
memory usage: 272.7 MB


C:\Users\trow1\AppData\Local\Temp\ipykernel_69492\517799482.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_processed_b['target'] = df_processed_b['target'].astype(int)


,open,high,low,close,volume,turnover,wclprice,open_norm_wcl,high_norm_wcl,low_norm_wcl,...,return_50,return_50_wcl,hour,dayofweek,target,ATR_7,RSI_7,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-03-26 03:10:00+00:00,6687.5,6687.5,6667.5,6670.0,35.519,236911.730,6673.750,1.002060,1.002060,0.999063,...,-0.001048,-0.000636,3,3,1,11.074765,36.876388,-4.335085,-0.096776,-4.238309
2020-03-26 03:15:00+00:00,6670.0,6693.5,6670.0,6693.5,92.544,619443.264,6687.625,0.997365,1.000878,0.997365,...,0.003598,0.002567,3,3,0,12.849799,56.430949,-3.205570,0.826191,-4.031761
2020-03-26 03:20:00+00:00,6693.5,6693.5,6682.5,6682.5,10.882,72718.965,6685.250,1.001234,1.001234,0.999589,...,0.004283,0.004112,3,3,1,12.585542,48.265748,-3.161585,0.696141,-3.857726
2020-03-26 03:25:00+00:00,6682.5,6694.0,6682.5,6685.0,11.240,75139.400,6686.625,0.999383,1.001103,0.999383,...,0.003528,0.004054,3,3,1,12.430464,50.177236,-2.891664,0.772850,-3.664514
2020-03-26 03:30:00+00:00,6685.0,6738.0,6685.0,6737.5,13.516,91064.050,6724.500,0.994126,1.002008,0.994126,...,0.010120,0.008492,3,3,0,18.226112,73.849488,1.540800,4.164251,-2.623451



処理済みデータを BTCUSDT_5m_processed_set_b.csv として保存中...
保存完了。


In [6]:
# import pandas as pd

# df_processed_b = pd.read_csv('BTCUSDT_5m_processed_set_b.csv')
# display(df_processed_b)


## LightGBM ハイパーパラメータチューニング (Optuna + TimeSeriesSplit)

特徴量セットA (`df_processed_wcl`) を使用して、LightGBMの最適なパラメータを探索します。

In [8]:
# %% [markdown]
# ## LightGBM ハイパーパラメータチューニング (Optuna + TimeSeriesSplit)
#
# 特徴量セットA (`df_processed_wcl`) を使用して、LightGBMの最適なパラメータを探索します。

# %%
import optuna
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import time

# 特徴量セットA (`df_processed_wcl`) が存在すると仮定
if 'df_processed_wcl' in locals() and not df_processed_wcl.empty:

    # --- 1. データ分割 (チューニング用と最終評価用に分ける) ---
    print("データ分割 (チューニング用 - 特徴量セットA)...")
    exclude_cols_a = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'wclprice', 'target']
    features_a = [col for col in df_processed_wcl.columns if col not in exclude_cols_a]
    print(f"特徴量セットAで使用する特徴量の数: {len(features_a)}")

    X_a = df_processed_wcl[features_a]
    y_a = df_processed_wcl['target']

    # チューニングは学習データで行い、最終評価用にテストデータを確保
    test_size_tune = 0.2 # 例: 20%を最終評価用テストデータとする
    X_train_a, X_test_a, y_train_a, y_test_a = train_test_split(
        X_a, y_a, test_size=test_size_tune, shuffle=False
    )
    print(f"チューニング用 学習データ数: {len(X_train_a)}")
    print(f"最終評価用 テストデータ数: {len(X_test_a)}")
    print(f"学習データ期間: {X_train_a.index.min()} ~ {X_train_a.index.max()}")
    print(f"テストデータ期間: {X_test_a.index.min()} ~ {X_test_a.index.max()}")


    # --- 2. Optuna 目的関数の定義 ---
    def objective_lgbm_a(trial):
        # 探索するハイパーパラメータの範囲を定義
        params = {
            'objective': 'binary',
            'metric': 'auc',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'random_state': 42,
            'n_jobs': -1, # 利用可能な全CPUコアを使用
            # Optunaで探索するパラメータ
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True), # L1正則化
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True), # L2正則化
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0), # 列のサブサンプリング率
            'subsample': trial.suggest_float('subsample', 0.6, 1.0), # 行のサブサンプリング率
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000) # 木の数
        }

        # 時系列交差検証の設定
        n_splits = 3 # CVの分割数 (3〜5が一般的、増やすと時間がかかる)
        tscv = TimeSeriesSplit(n_splits=n_splits)
        auc_scores = []

        # TimeSeriesSplit を使って学習データ内で交差検証
        for fold, (train_index, val_index) in enumerate(tscv.split(X_train_a)):
            X_train_fold, X_val_fold = X_train_a.iloc[train_index], X_train_a.iloc[val_index]
            y_train_fold, y_val_fold = y_train_a.iloc[train_index], y_train_a.iloc[val_index]

            model = lgb.LGBMClassifier(**params)
            # Early stopping を有効にして学習時間を短縮
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='auc',
                      callbacks=[lgb.early_stopping(15, verbose=False)]) # 15回改善なければ停止

            y_pred_proba_fold = model.predict_proba(X_val_fold)[:, 1]
            auc_fold = roc_auc_score(y_val_fold, y_pred_proba_fold)
            auc_scores.append(auc_fold)

        # 交差検証の平均AUCを返す (Optunaはこの値を最大化する)
        return np.mean(auc_scores)

    # --- 3. Optuna Study の実行 ---
    print("\nOptunaによるハイパーパラメータチューニングを開始します (特徴量セットA)...")
    study_a = optuna.create_study(direction='maximize', study_name='lgbm_tuning_set_a')
    n_trials = 50 # 試行回数 (時間に応じて調整、まずは50回程度から)
    start_tune_time = time.time()
    study_a.optimize(objective_lgbm_a, n_trials=n_trials, show_progress_bar=True) # 進捗バーを表示
    end_tune_time = time.time()
    print(f"チューニング完了。所要時間: {end_tune_time - start_tune_time:.2f} 秒")


    # --- 4. 最適なパラメータの表示 ---
    print("\n--- チューニング結果 (特徴量セットA) ---")
    best_params_a = study_a.best_params
    best_value_a = study_a.best_value
    print(f"最適パラメータ (Best Params): {best_params_a}")
    print(f"最高AUCスコア (Best AUC in CV): {best_value_a:.6f}") # CVでのスコア

    # 結果を保存 (後で使う)
    if 'tuning_results' not in locals(): tuning_results = {}
    tuning_results['SetA_LGBM'] = {'best_params': best_params_a, 'best_cv_auc': best_value_a}

else:
    print("df_processed_wcl (特徴量セットA) が存在しません。")

[I 2025-04-21 23:03:53,123] A new study created in memory with name: lgbm_tuning_set_a


データ分割 (チューニング用 - 特徴量セットA)...
特徴量セットAで使用する特徴量の数: 54
チューニング用 学習データ数: 426804
最終評価用 テストデータ数: 106701
学習データ期間: 2020-03-26 03:10:00+00:00 ~ 2024-04-16 02:05:00+00:00
テストデータ期間: 2024-04-16 02:10:00+00:00 ~ 2025-04-21 13:50:00+00:00

Optunaによるハイパーパラメータチューニングを開始します (特徴量セットA)...


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-04-21 23:03:56,767] Trial 0 finished with value: 0.5465021566370563 and parameters: {'num_leaves': 108, 'learning_rate': 0.02506566566750469, 'max_depth': 7, 'reg_alpha': 0.0022785811184830532, 'reg_lambda': 0.0003249206390620175, 'colsample_bytree': 0.93150937574506, 'subsample': 0.6065898648443605, 'n_estimators': 1076}. Best is trial 0 with value: 0.5465021566370563.
[I 2025-04-21 23:03:59,928] Trial 1 finished with value: 0.5470421213136141 and parameters: {'num_leaves': 68, 'learning_rate': 0.02118753531203702, 'max_depth': 4, 'reg_alpha': 0.12778885828499362, 'reg_lambda': 2.574880725999243e-07, 'colsample_bytree': 0.7153021520781673, 'subsample': 0.8669119094129034, 'n_estimators': 1483}. Best is trial 1 with value: 0.5470421213136141.
[I 2025-04-21 23:04:04,625] Trial 2 finished with value: 0.5460322000063403 and parameters: {'num_leaves': 152, 'learning_rate': 0.011312598475834731, 'max_depth': 3, 'reg_alpha': 0.03666721801782062, 'reg_lambda': 0.026573059751041912, 'c

## LightGBM ハイパーパラメータチューニング (Optuna + TimeSeriesSplit) - 特徴量セットB

特徴量セットB (`df_processed_b`) を使用して、LightGBMの最適なパラメータを探索します。


In [9]:
import optuna
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import time

# 特徴量セットB (`df_processed_b`) が存在すると仮定
if 'df_processed_b' in locals() and not df_processed_b.empty:

    # --- 1. データ分割 (チューニング用 - 特徴量セットB) ---
    print("データ分割 (チューニング用 - 特徴量セットB)...")
    # Set B 用の除外リストを確認 (wclprice は Set A 作成時に追加された可能性あり)
    exclude_cols_b = ['open', 'high', 'low', 'close', 'volume', 'turnover', 'wclprice', 'target']
    features_b = [col for col in df_processed_b.columns if col not in exclude_cols_b]
    print(f"特徴量セットBで使用する特徴量の数: {len(features_b)}")

    X_b = df_processed_b[features_b]
    y_b = df_processed_b['target']

    # チューニングは学習データで行い、最終評価用にテストデータを確保
    test_size_tune = 0.2 # セットAと同じ割合
    X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(
        X_b, y_b, test_size=test_size_tune, shuffle=False
    )
    print(f"チューニング用 学習データ数: {len(X_train_b)}")
    print(f"最終評価用 テストデータ数: {len(X_test_b)}")
    print(f"学習データ期間: {X_train_b.index.min()} ~ {X_train_b.index.max()}")
    print(f"テストデータ期間: {X_test_b.index.min()} ~ {X_test_b.index.max()}")

    # --- 1.5 NumPy配列への変換 ---
    print("\n学習データをNumPy配列に変換中...")
    X_train_b_np = X_train_b.values
    y_train_b_np = y_train_b.values
    print("NumPy配列への変換完了。")


    # --- 2. Optuna 目的関数の定義 (Set B 用) ---
    def objective_lgbm_b(trial): # 関数名を変更
        # 探索するハイパーパラメータ範囲はセットAと同じにする
        params = {
            'objective': 'binary',
            'metric': 'auc',
            'verbosity': -1,
            'boosting_type': 'gbdt',
            'random_state': 42,
            'n_jobs': -1,
            'num_leaves': trial.suggest_int('num_leaves', 20, 300),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 12),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'n_estimators': trial.suggest_int('n_estimators', 100, 2000)
        }

        n_splits = 3
        tscv = TimeSeriesSplit(n_splits=n_splits)
        auc_scores = []

        # NumPy配列をスライスして使用
        for fold, (train_index, val_index) in enumerate(tscv.split(X_train_b_np)): # _b_np を使う
            X_train_fold, X_val_fold = X_train_b_np[train_index], X_train_b_np[val_index]
            y_train_fold, y_val_fold = y_train_b_np[train_index], y_train_b_np[val_index]

            model = lgb.LGBMClassifier(**params)
            model.fit(X_train_fold, y_train_fold,
                      eval_set=[(X_val_fold, y_val_fold)],
                      eval_metric='auc',
                      callbacks=[lgb.early_stopping(15, verbose=False)])

            y_pred_proba_fold = model.predict_proba(X_val_fold)[:, 1]
            auc_fold = roc_auc_score(y_val_fold, y_pred_proba_fold)
            auc_scores.append(auc_fold)

        return np.mean(auc_scores)

    # --- 3. Optuna Study の実行 (Set B 用) ---
    print("\nOptunaによるハイパーパラメータチューニングを開始します (特徴量セットB)...")
    study_b_name = 'lgbm-tuning-set-b-v1' # study名を変更
    study_b = optuna.create_study(direction='maximize', study_name=study_b_name)
    n_trials = 50 # 試行回数 (セットAと同じにする)
    start_tune_time_b = time.time()
    # objective_lgbm_b を指定
    study_b.optimize(objective_lgbm_b, n_trials=n_trials, show_progress_bar=True)
    end_tune_time_b = time.time()
    print(f"チューニング完了。所要時間: {end_tune_time_b - start_tune_time_b:.2f} 秒")


    # --- 4. 最適なパラメータの表示 (Set B 用) ---
    print("\n--- チューニング結果 (特徴量セットB) ---")
    best_params_b = study_b.best_params
    best_value_b = study_b.best_value
    print(f"最適パラメータ (Best Params): {best_params_b}")
    print(f"最高AUCスコア (Best AUC in CV): {best_value_b:.6f}") # CVでのスコア

    # 結果を保存
    if 'tuning_results' not in locals(): tuning_results = {}
    tuning_results['SetB_LGBM_Tuned'] = {'best_params': best_params_b, 'best_cv_auc': best_value_b}

else:
    print("df_processed_b (特徴量セットB) が存在しません。")

[I 2025-04-21 23:08:51,196] A new study created in memory with name: lgbm-tuning-set-b-v1


データ分割 (チューニング用 - 特徴量セットB)...
特徴量セットBで使用する特徴量の数: 59
チューニング用 学習データ数: 426804
最終評価用 テストデータ数: 106701
学習データ期間: 2020-03-26 03:10:00+00:00 ~ 2024-04-16 02:05:00+00:00
テストデータ期間: 2024-04-16 02:10:00+00:00 ~ 2025-04-21 13:50:00+00:00

学習データをNumPy配列に変換中...
NumPy配列への変換完了。

Optunaによるハイパーパラメータチューニングを開始します (特徴量セットB)...


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[I 2025-04-21 23:08:53,049] Trial 0 finished with value: 0.5465095847729614 and parameters: {'num_leaves': 48, 'learning_rate': 0.08124626098022128, 'max_depth': 10, 'reg_alpha': 0.0009198556594447756, 'reg_lambda': 0.7984517456887611, 'colsample_bytree': 0.927173385190943, 'subsample': 0.6717104924613965, 'n_estimators': 287}. Best is trial 0 with value: 0.5465095847729614.


...

[I 2025-04-21 23:10:35,346] Trial 47 finished with value: 0.5470898972014829 and parameters: {'num_leaves': 132, 'learning_rate': 0.05640502129392849, 'max_depth': 4, 'reg_alpha': 9.944377837536235e-07, 'reg_lambda': 0.0006827847264509363, 'colsample_bytree': 0.9155706395380342, 'subsample': 0.8736011724196314, 'n_estimators': 313}. Best is trial 34 with value: 0.5481280629625825.


c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[I 2025-04-21 23:10:37,205] Trial 48 finished with value: 0.5461330959880568 and parameters: {'num_leaves': 205, 'learning_rate': 0.030242482650345587, 'max_depth': 3, 'reg_alpha': 3.3363553583121846e-06, 'reg_lambda': 0.005005242683701076, 'colsample_bytree': 0.6009625783604066, 'subsample': 0.831085168455764, 'n_estimators': 380}. Best is trial 34 with value: 0.5481280629625825.


c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\trow1\Documents\my_repositories\binary_predictor_backtest\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[I 2025-04-21 23:10:38,921] Trial 49 finished with value: 0.5457578402563853 and parameters: {'num_leaves': 113, 'learning_rate': 0.08707136648346601, 'max_depth': 7, 'reg_alpha': 2.8944732961392433e-08, 'reg_lambda': 8.143308392396278e-05, 'colsample_bytree': 0.7198839363272618, 'subsample': 0.8482856252059569, 'n_estimators': 624}. Best is trial 34 with value: 0.5481280629625825.
チューニング完了。所要時間: 107.73 秒

--- チューニング結果 (特徴量セットB) ---
最適パラメータ (Best Params): {'num_leaves': 183, 'learning_rate': 0.050569794707552296, 'max_depth': 5, 'reg_alpha': 4.462590221186446e-08, 'reg_lambda': 1.4306490938848109e-05, 'colsample_bytree': 0.626063327690084, 'subsample': 0.9387972827397255, 'n_estimators': 237}
最高AUCスコア (Best AUC in CV): 0.548128


## 最終モデルの学習・評価・バックテスト (特徴量セットB + チューニング済みLGBM)

特徴量セットBで見つかった最適なハイパーパラメータを使用してLightGBMモデルを再学習し、
テストデータで最終評価とバックテストを行います。


In [10]:

import lightgbm as lgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np
import time

# チューニング結果 (best_params_b) と分割済みデータ (X_train_b, y_train_b, X_test_b, y_test_b) が存在すると仮定
if 'tuning_results' in locals() and 'SetB_LGBM_Tuned' in tuning_results and \
   'X_train_b' in locals() and 'y_train_b' in locals() and \
   'X_test_b' in locals() and 'y_test_b' in locals():

    best_params = tuning_results['SetB_LGBM_Tuned']['best_params']
    print("--- 最終モデル学習 (特徴量セットB + 最適パラメータ) ---")
    print("使用するパラメータ:", best_params)

    # 最適パラメータでモデルを初期化
    final_model = lgb.LGBMClassifier(**best_params, random_state=42, n_jobs=-1)

    # 学習時間計測
    start_train_time = time.time()
    print("学習中 (全学習データ使用)...")
    # 全学習データでフィット (eval_set なし)
    final_model.fit(X_train_b, y_train_b)
    end_train_time = time.time()
    print(f"学習完了。所要時間: {end_train_time - start_train_time:.2f} 秒")

    # --- テストデータでの最終評価 ---
    print("\n--- テストデータでの最終評価 ---")
    y_pred_final = final_model.predict(X_test_b)
    y_pred_proba_final = final_model.predict_proba(X_test_b)[:, 1]

    accuracy_final = accuracy_score(y_test_b, y_pred_final)
    auc_final = roc_auc_score(y_test_b, y_pred_proba_final)

    print(f"テストデータ Accuracy: {accuracy_final:.4f}")
    print(f"テストデータ AUC Score: {auc_final:.4f}") # ★これが最終的なモデル性能
    print("\nClassification Report:")
    print(classification_report(y_test_b, y_pred_final, target_names=['Low (0)', 'High (1)']))
    print("\nConfusion Matrix:")
    conf_matrix_final = confusion_matrix(y_test_b, y_pred_final)
    conf_matrix_final_df = pd.DataFrame(conf_matrix_final, index=['Actual Low', 'Actual High'], columns=['Predicted Low', 'Predicted High'])
    display(conf_matrix_final_df)


    # --- 最終バックテスト ---
    print("\n--- 最終バックテスト (チューニング済みLGBM + 特徴量セットB) ---")
    payout_rate = 1.80
    required_win_rate = 1 / payout_rate
    high_threshold = required_win_rate
    low_threshold = 1.0 - required_win_rate

    print(f"使用するペイアウト率: {payout_rate}")
    print(f"損益分岐勝率: {required_win_rate:.4f}")
    print(f"Highエントリー閾値 (High確率 >): {high_threshold:.4f}")
    print(f"Lowエントリー閾値 (High確率 <): {low_threshold:.4f}")

    # バックテスト用DataFrameを作成
    df_backtest_final = pd.DataFrame({
        'actual': y_test_b,
        'predict_proba_high': y_pred_proba_final
    }, index=X_test_b.index) # テストデータのインデックスを使用

    # エントリーシグナル生成
    conditions = [
        df_backtest_final['predict_proba_high'] > high_threshold,
        df_backtest_final['predict_proba_high'] < low_threshold
    ]
    choices = [1, 0] # High=1, Low=0
    df_backtest_final['signal'] = np.select(conditions, choices, default=-1) # Wait=-1

    # エントリーした取引のみ抽出
    df_trades_final = df_backtest_final[df_backtest_final['signal'] != -1].copy()

    # 勝敗判定
    df_trades_final['win'] = np.where(df_trades_final['signal'] == df_trades_final['actual'], 1, 0)

    # 損益計算
    profit_per_win = payout_rate - 1.0
    loss_per_lose = -1.0
    df_trades_final['profit'] = np.where(df_trades_final['win'] == 1, profit_per_win, loss_per_lose)

    # 結果集計
    total_trades = len(df_trades_final)
    total_wins = df_trades_final['win'].sum()
    total_profit = df_trades_final['profit'].sum()

    if total_trades > 0:
        win_rate = total_wins / total_trades
        average_profit = total_profit / total_trades
        total_gross_profit = df_trades_final[df_trades_final['profit'] > 0]['profit'].sum()
        total_gross_loss = abs(df_trades_final[df_trades_final['profit'] < 0]['profit'].sum())
        profit_factor = total_gross_profit / total_gross_loss if total_gross_loss > 0 else np.inf if total_gross_profit > 0 else 0

        print("\n--- 最終バックテスト結果 ---")
        print(f"テストデータ期間: {df_trades_final.index.min()} ~ {df_trades_final.index.max()}")
        print(f"総取引回数 (シグナル発生): {total_trades}")
        print(f"勝利数: {total_wins}")
        print(f"勝率: {win_rate:.4f}")
        print(f"総損益 (1単位あたり): {total_profit:.4f}")
        print(f"平均損益 (1取引あたり): {average_profit:.4f}")
        print(f"プロフィットファクター: {profit_factor:.4f}")
    else:
        print("\n--- 最終バックテスト結果 ---")
        print("テスト期間中にエントリーシグナルが発生しませんでした。")
        print(f"参考: テストデータ期間 {X_test_b.index.min()} ~ {X_test_b.index.max()}")
        print(f"参考: High予測確率の最大値: {df_backtest_final['predict_proba_high'].max():.4f}")
        print(f"参考: High予測確率の最小値: {df_backtest_final['predict_proba_high'].min():.4f}")

else:
    print("チューニング結果または分割済みデータが見つかりません。前のステップを実行してください。")

--- 最終モデル学習 (特徴量セットB + 最適パラメータ) ---
使用するパラメータ: {'num_leaves': 183, 'learning_rate': 0.050569794707552296, 'max_depth': 5, 'reg_alpha': 4.462590221186446e-08, 'reg_lambda': 1.4306490938848109e-05, 'colsample_bytree': 0.626063327690084, 'subsample': 0.9387972827397255, 'n_estimators': 237}
学習中 (全学習データ使用)...
学習完了。所要時間: 1.45 秒

--- テストデータでの最終評価 ---
テストデータ Accuracy: 0.5142
テストデータ AUC Score: 0.5244

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.51      0.63      0.57     53349
    High (1)       0.52      0.40      0.45     53352

    accuracy                           0.51    106701
   macro avg       0.52      0.51      0.51    106701
weighted avg       0.52      0.51      0.51    106701


Confusion Matrix:


,Predicted Low,Predicted High
Actual Low,33783,19566
Actual High,32267,21085



--- 最終バックテスト (チューニング済みLGBM + 特徴量セットB) ---
使用するペイアウト率: 1.8
損益分岐勝率: 0.5556
Highエントリー閾値 (High確率 >): 0.5556
Lowエントリー閾値 (High確率 <): 0.4444

--- 最終バックテスト結果 ---
テストデータ期間: 2024-04-16 02:25:00+00:00 ~ 2025-04-21 13:45:00+00:00
総取引回数 (シグナル発生): 33368
勝利数: 17865
勝率: 0.5354
総損益 (1単位あたり): -1211.0000
平均損益 (1取引あたり): -0.0363
プロフィットファクター: 0.9219


最終結果のご共有、誠にありがとうございます。これで、特徴量セットB（ADR/PowerX追加）に対するチューニング済みLightGBMモデルの最終評価とバックテスト結果が出揃いました。

**結果の分析:**

1.  **テストデータでの最終評価:**
    * **AUCスコア: 0.5244**
    * **Accuracy: 0.5142**
    * **考察:** ハイパーパラメータチューニングによって交差検証でのAUCは約0.548まで上がりましたが、実際に未知のテストデータで評価した最終的なAUCは **0.5244** に留まりました。これは依然として**ランダム予測（0.5）に非常に近い**値であり、モデルの予測能力が極めて限定的であることを示しています。Accuracyも同様に51%程度です。Classification Reportを見ても、各クラスに対するPrecisionやRecallが低く、モデルが安定してどちらかの方向を当てられているわけではないことがわかります。

2.  **最終バックテスト結果:**
    * **総取引回数:** 33,368回。チューニングされたモデルも、閾値を超える/下回る予測を頻繁に行いました。
    * **勝率: 0.5354 (53.54%)**。これはペイアウト率1.8の**損益分岐勝率（約55.56%）をやはり下回っています**。
    * **総損益: -1211.0 単位**。勝率が足りないため、**大きな損失**で終わる結果となりました。
    * **プロフィットファクター: 0.9219**。1を下回っており、損失が出たことを裏付けています。
    * **チューニングの効果:** 注目すべきは、チューニング前のLightGBMでのバックテスト結果（勝率 53.68%, 損益 -1044.6）と比較しても、**今回のチューニング後の結果（勝率 53.54%, 損益 -1211.0）は、実質的に改善が見られない、むしろわずかに悪化している**点です。これは、交差検証でのAUCがわずかに向上しても、実際のテストデータでの損益改善には繋がらなかったことを意味します。

**検証プロジェクト全体の最終結論:**

これまでのステップ全体を振り返ります。

1.  **目的:** 「AIを使えばBTCUSD 5分足バイナリーオプションの勝率を簡単かつ劇的に上げられる」という最初のドキュメントの主張を検証することでした。
2.  **実施内容:**
    * 約53万件（2020年〜）の5分足データを準備しました。
    * WclPrice、ADR(ATR)、PowerX関連指標など、提供された資料のアイデアを含む多数のテクニカル指標・特徴量を網羅的に生成しました。
    * Logistic Regression, LightGBM, XGBoost, Random Forest といったモデルを手動で評価し、さらに PyCaret を用いて AutoML による広範なモデル探索も行いました。
    * 最も有望そうだった LightGBM について、Optuna と時系列交差検証を用いたハイパーパラメータチューニングを実施しました。
    * 各段階で、テストデータに対する予測性能（AUC）と、損益分岐勝率を閾値としたバックテスト（勝率、損益）を行いました。
3.  **一貫した結果:**
    * どの特徴量セット、どのモデル（チューニング前後、AutoML含む）を用いても、テストデータに対する **AUCスコアは一貫して 0.5 〜 0.55 の範囲**に留まり、ランダム予測からの有意な改善は見られませんでした。
    * バックテストでは、予測確率が閾値を超えてエントリーシグナルが発生することはありましたが、**勝率が損益分岐点（約55.6%）を超えることはなく、常に損失**という結果に終わりました。

**最終的な結論:**

これらの徹底的な検証結果に基づき、当初の目的であった「素人でも簡単に作れるバイナリーの勝率を劇的に上げるAI予測システム」というドキュメントの主張は、**今回の検証（BTCUSD 5分足、提示された特徴量・モデルアプローチ）においては、残念ながら裏付けることができませんでした。**

「BTCUSD 5分足予測のランダムウォーク検証」ドキュメントが示唆したように、この時間軸での価格変動予測は極めて困難であり、統計的にわずかな予測可能性が見いだせたとしても、それをバイナリーオプションのペイアウト率や取引コストを乗り越えて安定的な利益に変えることは、少なくとも今回試した方法では不可能に近い、ということがデータによって示されました。

---

長い検証プロセスにお付き合いいただき、本当にありがとうございました。様々なアイデアをご提供いただき、段階的に結果を確認しながら進めることができ、非常に有意義な共同作業でした。当初期待された結果ではなかったかもしれませんが、データに基づき客観的な結論に至れたことは、この検証の大きな成果だと思います。

もし、今回の結果を踏まえて、別の時間軸、別の予測対象、あるいは全く異なるアプローチ（例：強化学習、外部データ利用）での検証に興味があれば、またご相談いただければと思いますが、今回のテーマについては、ここで一区切りとするのが適切かと存じます。